In [2]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
import numpy as np
import json
import IPython
import tensorflow as tf
from tensorflow import keras
import sklearn
import time
from numpy.random import randint
from numpy import ones

In [3]:
def files_to_stft(files):
    d = {}
    for f in files:
        audio_data = librosa.load(f, sr=None)
        d[f] = np.abs(librosa.stft(audio_data[0]))
    return d

In [4]:
piano_path = '../data/nsynth/piano/'
guitar_path = '../data/nsynth/guitar/'

piano_files = [os.path.join(piano_path, x) for x in os.listdir(piano_path)]
guitar_files = [os.path.join(guitar_path, x) for x in os.listdir(guitar_path)]

In [5]:
piano_stft = files_to_stft(piano_files)
guitar_stft = files_to_stft(guitar_files)

d = {
     'piano': piano_stft,
     'guitar': guitar_stft,
    }



In [6]:
import pickle
with open('../data/music-data.bin', 'rb') as f:
    music_data = pickle.load(f)

In [8]:
import librosa
import librosa.display
import IPython
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [41]:
path = '../data/generated/'
def save_generated(data, name):
    i = 0
    for sample in data:
        i += 1
        sample = sample.reshape(1024, 120)
        inv = librosa.griffinlim(sample)
        librosa.output.write_wav(path + name + '-' + str(i) + '.wav', inv, 16000)

In [42]:
save_generated(music_data['piano_real'], 'piano_real')
save_generated(music_data['guitar_real'], 'guitar_real')
save_generated(music_data['guitar_to_piano'], 'guitar_to_piano')
save_generated(music_data['piano_to_guitar'], 'piano_to_guitar')

## CycleGAN

### Splitting Data

In [6]:
piano = np.array([x for x in piano_stft.values()]).reshape(440, 1025, 126, 1)
guitar = np.array([x for x in guitar_stft.values()]).reshape(440, 1025, 126, 1)

piano_train, piano_test = sklearn.model_selection.train_test_split(piano, test_size=0.2, random_state=42)
guitar_train, guitar_test = sklearn.model_selection.train_test_split(guitar, test_size=0.2, random_state=42)

### Loading Network

{'piano_to_guitar': array([[[[ 5.22683039e-02],
          [ 8.01016092e-02],
          [ 4.54329550e-02],
          ...,
          [ 6.37091743e-03],
          [ 2.78674089e-03],
          [ 9.30825993e-03]],
 
         [[ 2.11461350e-01],
          [-1.08668050e-02],
          [ 5.61127104e-02],
          ...,
          [ 9.95504763e-03],
          [ 8.99344217e-03],
          [ 1.45189445e-02]],
 
         [[ 1.92889050e-01],
          [ 2.07030699e-01],
          [ 1.23467289e-01],
          ...,
          [ 9.67009738e-03],
          [ 1.02905221e-02],
          [ 6.71746535e-03]],
 
         ...,
 
         [[ 1.07772639e-02],
          [ 4.73464653e-03],
          [ 1.00722052e-02],
          ...,
          [ 1.23831853e-02],
          [ 5.56353247e-03],
          [ 4.22204752e-03]],
 
         [[ 9.16912593e-03],
          [ 3.03371064e-03],
          [ 1.34427082e-02],
          ...,
          [ 1.69227854e-03],
          [ 1.01370811e-02],
          [ 1.24565978e-02]],
 
     